# Preprocessing
## 00. Combine the meteo_data 

The meteorological variables were only possible to download from the [complete eartH2Observe data catalogue](https://wci.earth2observe.eu/thredds/catalog-earth2observe-model-wrr2.html). The dataset was only avalibale in daily daily and 3-hourly timesteps.

To manage the large size of the dataset and to facilitate practical downloading, I downloaded the NETCDF dataset in two phases, spanning the years 1979-1989 and 1989-2014. 

In this code we first combine the two data files for each meteo variable and then resample them to monthly timestep.

## Importing the packages

In [1]:
# Import packages
from os.path import join as pjoin
from scipy.io import netcdf
import numpy as np
import os

import matplotlib.pyplot as plt
import xarray as xr
# Any import of metpy will activate the accessors
from metpy.cbook import get_test_data
import pandas as pd

## Set the directory and read the netCDF file

In [2]:
directory = 'data'
os.chdir(directory)

In [23]:
folder = "meteo_daily"
output_folder = "meteo"
file = "rain"


path_nc_file1 = pjoin(folder, "{}_daily_{}.nc".format(file, 1))
path_nc_file2 = pjoin(folder, "{}_daily_{}.nc".format(file, 2))

In [24]:
# read the first half of the ncdffile
ds1 = xr.open_dataset(path_nc_file1)

#Select the variable. In this case we have only one variable in  each nc file, so we select one.
ds_variable1 = ds1[list(ds1.variables)[0]]


# read the second half of the ncdffile
ds2 = xr.open_dataset(path_nc_file2)
ds_variable2 = ds2[list(ds2.variables)[0]]


# Combine the two datasets along the time dimension
combined_ds = xr.concat([ds_variable1, ds_variable2], dim='time')

# Set the coordinate reference system (CRS)
crs = "EPSG:4326"
combined_ds = combined_ds.assign_attrs(crs=crs)


# Remove duplicate dates and keep the first occurrence
combined_ds = combined_ds.drop_duplicates('time', keep='first')

#Convert it into df
df_combined = combined_ds.to_dataframe()

#Convert it into df
df_combined = combined_ds.to_dataframe()


# Reset the index of the DataFrame
df = df_combined.reset_index()

# Convert the 'time' column to a datetime object
df['time'] = pd.to_datetime(df['time'])

# Set 'time', 'lat', 'lon' as the DataFrame index
df = df.set_index(['time', 'lat', 'lon'])

# Resample the DataFrame to monthly frequency and calculate the mean
df_monthly = df.groupby([pd.Grouper(freq='M', level='time'), 'lat', 'lon']).mean()

# Reset the index of the monthly DataFrame
df_monthly = df_monthly.reset_index()

# Set 'time', 'lat', 'lon' as the DataFrame index
df_monthly = df_monthly.set_index(['time', 'lat', 'lon'])
da_monthly = df_monthly.to_xarray()
#da_monthly = da_monthly["Tair"]

In [25]:
path_output_file = pjoin(output_folder, "{}.nc".format(file))
# Save the combined dataset to a new NetCDF file
da_monthly.to_netcdf(path_output_file)